In [1]:
library(reticulate) 
library(Seurat)

Attaching SeuratObject



In [2]:
#### user specified

data_folder = "D:/analyze_Pearson_residuals/"
data_subfolder = "lupus"

data_path  <- paste0 ( data_folder, data_subfolder )

In [3]:
# output data
residual_variance_pkl = "SCT_v2_residual_variance.pkl"
Pearson_residuals_pkl = "SCT_v2_Pearson_residuals_top_2000.pkl"

# input data
gene_array_pkl  = "gene_array.pkl"
cell_array_pkl  = "cell_array.pkl"
counts_pkl = "counts_scipy_csc.pkl"



# paths: output data
residual_variance_dsn    <- paste0 ( data_path, '/', residual_variance_pkl )
Pearson_residuals_dsn    <- paste0 ( data_path, '/', Pearson_residuals_pkl )

# paths: input data
gene_array_dsn    <- paste0 ( data_path, '/', gene_array_pkl )
cell_array_dsn   <- paste0 ( data_path, '/', cell_array_pkl )
counts_dsn    <- paste0 ( data_path, '/', counts_pkl )

In [4]:
gene_array = py_load_object ( gene_array_dsn )
print ( dim ( gene_array ) )
print ( head ( gene_array ) )

[1] 10619
[1] "ENSG00000244734" "ENSG00000188536" "ENSG00000167996" "ENSG00000206172"
[5] "ENSG00000087086" "ENSG00000187608"


In [5]:
cell_array = py_load_object ( cell_array_dsn )
print ( dim ( cell_array ) )
print ( head ( cell_array ) )

[1] 26820
[1] "AAACATACAATGCC-1" "AAACATACATTTCC-1" "AAACATACCAGAAA-1" "AAACATACCAGCTA-1"
[5] "AAACATACCATGCA-1" "AAACATACCTCGCT-1"


In [6]:
df_counts = py_load_object( counts_dsn ) 

rownames(df_counts) <- gene_array
colnames(df_counts) <- cell_array

print ( dim ( df_counts ) )
print ( head (df_counts [, c(1:5)] ) )

[1] 10619 26820
6 x 5 sparse Matrix of class "dgCMatrix"
                AAACATACAATGCC-1 AAACATACATTTCC-1 AAACATACCAGAAA-1
ENSG00000244734                .                .                .
ENSG00000188536                .                .                .
ENSG00000167996               32              388              602
ENSG00000206172                .                .                .
ENSG00000087086                .              279              152
ENSG00000187608                .                .                .
                AAACATACCAGCTA-1 AAACATACCATGCA-1
ENSG00000244734                .                .
ENSG00000188536                .                .
ENSG00000167996              387               16
ENSG00000206172                .                .
ENSG00000087086              425               48
ENSG00000187608                .                .


In [7]:
seurat_object <-  CreateSeuratObject(counts = df_counts , project = data_subfolder )
print ( dim ( seurat_object ) )

[1] 10619 26820


In [8]:
seurat_object <- SCTransform(seurat_object, vst.flavor="v2", method = "glmGamPoi", variable.features.n=2000 )

gene_list = seurat_object@assays$SCT@data@Dimnames[1]
gene_vector = unlist ( gene_list )

residual_variance = seurat_object@assays$SCT@SCTModel.list$model1@feature.attributes$residual_variance
df_residual_variance <- data.frame( residual_variance )
row.names( df_residual_variance ) <- gene_vector
print ( dim ( df_residual_variance ) ) 

vst.flavor='v2' set, setting model to use fixed slope and exclude poisson genes.

Calculating cell attributes from input UMI matrix: log_umi

Total Step 1 genes: 10609

Total overdispersed genes: 10429

Excluding 180 genes from Step 1 because they are not overdispersed.

Variance stabilizing transformation of count matrix of size 10619 by 26820

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 2000 genes, 5000 cells



  |======================================================================| 100%


Setting estimate of  247 genes to inf as theta_mm/theta_mle < 1e-3

# of step1 poisson genes (variance < mean): 0

# of low mean genes (mean < 0.001): 0

Total # of Step1 poisson genes (theta=Inf; variance < mean): 247

Total # of poisson genes (theta=Inf; variance < mean): 182

Calling offset model for all 182 poisson genes

Found 295 outliers - those will be ignored in fitting/regularization step


Ignoring theta inf genes

Replacing fit params for 182 poisson genes by theta=Inf

Setting min_variance based on median UMI:  0.04

Second step: Get residuals using fitted parameters for 10619 genes



  |======================================================================| 100%


Computing corrected count matrix for 10619 genes



  |======================================================================| 100%


Calculating gene attributes

Wall clock passed: Time difference of 1.186374 mins

Determine variable features

Place corrected count matrix in counts slot

Centering data matrix

Set default assay to SCT



[1] 10619     1


In [9]:
SCT_assay =  seurat_object@assays$SCT
residuals_top_features = SCT_assay@scale.data
print ( dim ( residuals_top_features ) )

[1]  2000 26820


In [10]:
py_save_object ( df_residual_variance, residual_variance_dsn )
py_save_object ( as.data.frame ( residuals_top_features ), Pearson_residuals_dsn )